In [1]:
#pip install biopython

!pip install biopython

In [3]:
from Bio.PDB import PDBParser, DSSP
import pandas as pd
import os

# Function to perform DSSP and return DataFrame
def run_dssp(pdb_file_path):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure(os.path.basename(pdb_file_path).split('.')[0], pdb_file_path)
    model = structure[0]
    dssp = DSSP(model, pdb_file_path, dssp='mkdssp', acc_array='Sander')

    # Prepare data for DataFrame
    data = [(dssp[key][0],     # dssp_index
             key[0],            # chain_id
             dssp[key][1],     # aa (amino acid)
             dssp[key][2],     # eight_hot_ss (secondary structure in eight definitions)
             dssp[key][2].translate(str.maketrans('HGIEBTS-', 'HHHEECCC')),  # three_hot_ss (converted secondary structure)
             dssp[key][3],     # rsa (Relative ASA)
             dssp[key][4],     # phi (Phi angle)
             dssp[key][5])     # psi (Psi angle)
            for key in dssp.keys()]

    df = pd.DataFrame(data, columns=['dssp_index', 'chain', 'aa', 'eight_hot_ss', 'three_hot_ss', 'rsa', 'phi', 'psi'])
    return df

# Directory containing PDB files
pdb_directory = "data"

# List all PDB files in the directory
pdb_files = [f for f in os.listdir(pdb_directory) if f.endswith('.pdb')]

# Given protein sequence
sequence = "NKVQMHRSEMRPKFFSEHIISILNPHCVV"

# Process each PDB file
for pdb_file in pdb_files:
    pdb_file_path = os.path.join(pdb_directory, pdb_file)
    df = run_dssp(pdb_file_path)

    # Filter DataFrame for the given sequence
    filtered_df = df[df['aa'].isin(list(sequence))]

    # Display RSA values for the given sequence
    print(f"RSA values for sequence {sequence} in {pdb_file}:")
    display(filtered_df[['dssp_index', 'chain', 'aa', 'rsa']])


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data'